In [1]:
# Sheet properties

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [3]:
# Imports

In [4]:
import sys
sys.path.append('/home/rcendre/classification')
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from numpy import array, ones, maximum
from sklearn.preprocessing import LabelEncoder
from toolbox.models.builtin import Applications
from toolbox.classification.common import IO, Tools, Folds
from toolbox.classification.parameters import Settings, Dermatology
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import cv2
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Parameters

In [6]:
validation = 4
settings = Settings.get_default_dermatology()

In [7]:
# Inputs

In [8]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)

In [9]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

In [10]:
# Additionnal

In [11]:
advanced = {'batch_size': 8,
            'epochs' : 15,
            'rotation_range': 45,
            'width_shift_range': 0.2,
            'height_shift_range': 0.2,
            'horizontal_flip': True,
            'vertical_flip': True}

model = Applications.get_fine_tuning(3, 0, -1, architecture='ResNet', pooling='avg', additional=advanced)

In [12]:
# Models

In [13]:
model = Tools.fit(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model)
model.save('FineTuned')

In [14]:
# Visualisation

In [ ]:
model.load('FineTuned')
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(-1).output, model.output])

IMAGE_INDEX = 0
CAT_CLASS_INDEX = 2
image_path = inputs['Datum'][IMAGE_INDEX]
img = tf.keras.preprocessing.image.load_img(image_path)
img = tf.keras.preprocessing.image.img_to_array(img)
img_size = img.shape

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CAT_CLASS_INDEX]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), img_size)
cam = maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

cv2.imwrite(f'cam_{IMAGE_INDEX}.png', output_image)

Found 4073 images belonging to 3 classes.
Pre-training...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 510 steps
Epoch 1/15
